In [28]:
# Cell 1: Import libraries
from google.cloud import bigquery
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd

In [35]:
# Setup: Load config and convert relative paths to absolute
import os
import json

# Find project root (2 levels up from notebooks/)
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '../..'))

# Load config
config_path = os.path.join(PROJECT_ROOT, 'data-pipeline/configs/pipeline_config.json')
with open(config_path, 'r') as f:
    config = json.load(f)

# Convert relative config path to absolute path
RAW_DATA_DIR = os.path.join(PROJECT_ROOT, config['pipeline_config']['input_path'])
os.makedirs(RAW_DATA_DIR, exist_ok=True)
print(f"✓ Project root: {PROJECT_ROOT}")
print(f"✓ Data directory: {RAW_DATA_DIR}")

✓ Project root: /Users/Admin/Desktop/lab-lens
✓ Data directory: /Users/Admin/Desktop/lab-lens/data-pipeline/data/raw


In [30]:
# Cell 2: Authenticate
from google.cloud import bigquery
from google_auth_oauthlib.flow import InstalledAppFlow

flow = InstalledAppFlow.from_client_config(
    {
        "installed": {
            "client_id": "764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com",
            "client_secret": "d-FL95Q19q7MQmFpd7hHD0Ty",
            "auth_uri": "https://accounts.google.com/o/oauth2/auth",
            "token_uri": "https://oauth2.googleapis.com/token",
            "redirect_uris": ["http://localhost"]
        }
    },
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

credentials = flow.run_local_server(port=8081, prompt='consent')
client = bigquery.Client(project='regal-bonito-455919-u3', credentials=credentials)
print("✓ Connected!")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8081%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=d0eQzM1dc3ZMZbjQfSwnzgDw3FIyZ7&prompt=consent&access_type=offline
✓ Connected!


In [31]:
# Cell 3: Test query
test_query = """
SELECT COUNT(*) as count
FROM `physionet-data.mimiciii_clinical.admissions`
"""

df = client.query(test_query).to_dataframe()
print(f"Total admissions: {df['count'][0]}")

Total admissions: 58976


In [32]:
# Cell to check available tables
# List all tables in the mimiciii_clinical dataset
tables = client.list_tables('physionet-data.mimiciii_clinical')
print("Available tables:")
for table in tables:
    print(f"  - {table.table_id}")

Available tables:
  - admissions
  - callout
  - caregivers
  - chartevents
  - cptevents
  - d_cpt
  - d_icd_diagnoses
  - d_icd_procedures
  - d_items
  - d_labitems
  - datetimeevents
  - diagnoses_icd
  - drgcodes
  - icustays
  - inputevents_cv
  - inputevents_mv
  - labevents
  - microbiologyevents
  - outputevents
  - patients
  - prescriptions
  - procedureevents_mv
  - procedures_icd
  - services
  - transfers


In [33]:
# Check for other datasets that might contain notes
datasets = list(client.list_datasets('physionet-data'))
for dataset in datasets:
    if 'mimic' in dataset.dataset_id.lower():
        print(f"Dataset: {dataset.dataset_id}")
        

Dataset: mimiciii_clinical
Dataset: mimiciii_derived
Dataset: mimiciii_notes
Dataset: mimiciii_notes_derived


In [36]:
# Enhanced query with demographics for bias detection
query = r"""
WITH discharge AS (
    SELECT 
        n.hadm_id,
        n.subject_id,
        REGEXP_REPLACE(n.text, r'\[\*\*[^\]]*\*\*\]', '') AS cleaned_text,
        LENGTH(n.text) as text_length
    FROM `physionet-data.mimiciii_notes.noteevents` n
    WHERE n.category = 'Discharge summary'
    LIMIT 5000
),
demographics AS (
    SELECT 
        p.subject_id,
        p.gender,
        a.ethnicity,
        a.insurance,
        a.language,
        a.marital_status,
        a.admission_type,
        DATETIME_DIFF(a.admittime, p.dob, YEAR) as age_at_admission
    FROM `physionet-data.mimiciii_clinical.patients` p
    JOIN `physionet-data.mimiciii_clinical.admissions` a 
        ON p.subject_id = a.subject_id
),
labs AS (
    SELECT 
        le.hadm_id,
        STRING_AGG(
            CONCAT(d.label, ': ', CAST(le.value AS STRING), ' ', le.valueuom,
            CASE WHEN le.flag = 'abnormal' THEN ' (!)' ELSE '' END), '; '
            LIMIT 20
        ) AS lab_summary,
        COUNT(*) as total_labs,
        SUM(CASE WHEN le.flag = 'abnormal' THEN 1 ELSE 0 END) AS abnormal_count
    FROM `physionet-data.mimiciii_clinical.labevents` le
    JOIN `physionet-data.mimiciii_clinical.d_labitems` d ON le.itemid = d.itemid
    GROUP BY le.hadm_id
),
diagnoses AS (
    SELECT 
        hadm_id,
        COUNT(*) as diagnosis_count,
        STRING_AGG(icd9_code, ', ' LIMIT 10) as top_diagnoses
    FROM `physionet-data.mimiciii_clinical.diagnoses_icd`
    GROUP BY hadm_id
)
SELECT 
    d.*,
    dem.gender,
    dem.ethnicity,
    dem.age_at_admission,
    dem.insurance,
    dem.language,
    dem.admission_type,
    l.lab_summary,
    l.total_labs,
    l.abnormal_count,
    diag.diagnosis_count,
    diag.top_diagnoses
FROM discharge d
LEFT JOIN demographics dem ON d.subject_id = dem.subject_id
LEFT JOIN labs l ON d.hadm_id = l.hadm_id
LEFT JOIN diagnoses diag ON d.hadm_id = diag.hadm_id
"""

print("Running comprehensive query with demographics...")
df_complete = client.query(query).to_dataframe()
print(f"✓ Loaded {len(df_complete)} records with full demographics")
print(f"✓ Average text length: {df_complete['text_length'].mean():.0f} characters")
print(f"✓ Columns: {df_complete.columns.tolist()}")

# Save complete dataset
output_path = os.path.join(RAW_DATA_DIR, 'mimic_discharge_labs.csv')
df_complete.to_csv(output_path, index=False)
print(f"✓ Saved: {output_path} ({os.path.getsize(output_path)/1024/1024:.2f} MB)")

Running comprehensive query with demographics...
✓ Loaded 9715 records with full demographics
✓ Average text length: 10164 characters
✓ Columns: ['hadm_id', 'subject_id', 'cleaned_text', 'text_length', 'gender', 'ethnicity', 'age_at_admission', 'insurance', 'language', 'admission_type', 'lab_summary', 'total_labs', 'abnormal_count', 'diagnosis_count', 'top_diagnoses']
✓ Saved: /Users/Admin/Desktop/lab-lens/data-pipeline/data/raw/mimic_discharge_labs.csv (91.40 MB)
